# MobileNet V3 Large

### Import libraries

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.applications.mobilenet_v3 import preprocess_input
print(f"TensorFlow Version: {tf.__version__}")

c:\Users\User\anaconda3\envs\tf1\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


TensorFlow Version: 2.10.1


In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


### Data Loading and Preprocessing

In [3]:
(ds_train, ds_val, ds_test), ds_info = tfds.load(
    'oxford_flowers102',
    split=['test', 'validation', 'train'],
    shuffle_files=True,
    as_supervised=True,  # Include labels
    with_info=True
)
print(f"Dataset Info:\n{ds_info}")
num_classes = ds_info.features['label'].num_classes
print(f"Number of classes: {num_classes}")

Dataset Info:
tfds.core.DatasetInfo(
    name='oxford_flowers102',
    full_name='oxford_flowers102/2.1.1',
    description="""
    The Oxford Flowers 102 dataset is a consistent of 102 flower categories commonly
    occurring in the United Kingdom. Each class consists of between 40 and 258
    images. The images have large scale, pose and light variations. In addition,
    there are categories that have large variations within the category and several
    very similar categories.
    
    The dataset is divided into a training set, a validation set and a test set. The
    training set and validation set each consist of 10 images per class (totalling
    1020 images each). The test set consists of the remaining 6149 images (minimum
    20 per class).
    
    Note: The dataset by default comes with a test size larger than the train size.
    For more info see this
    [issue](https://github.com/tensorflow/datasets/issues/3022).
    """,
    homepage='https://www.robots.ox.ac.uk/~vgg/da

In [4]:
IMG_SIZE = 224  # You can adjust this size
BATCH_SIZE = 64

def preprocess(image, label):
    # Resize and normalize images
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    # image = image / 255.0  # Normalize to [0,1]
    return image, label

ds_train = ds_train.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.shuffle(1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

ds_val = ds_val.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
ds_val = ds_val.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

ds_test = ds_test.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

### Use V3 base model

In [5]:
v3_model = keras.applications.MobileNetV3Large(
    weights='imagenet',     # Use 'imagenet' for pre-trained weights, or None
    include_top=False,      # Exclude the top classification layers
    input_shape=(224, 224, 3)
)

v3_model.trainable = False

### Wrap it layer

In [6]:
inputs = layers.Input(shape=(224, 224, 3))
x = preprocess_input(inputs)
x = v3_model(inputs, training=False)  # Set training=False if base_model is frozen
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(102, activation='softmax')(x)  # Adjust the number of classes
model = models.Model(inputs, x)
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 MobilenetV3large (Functiona  (None, 7, 7, 960)        2996352   
 l)                                                              
                                                                 
 global_average_pooling2d (G  (None, 960)              0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 102)               98022     
                                                                 
Total params: 3,094,374
Trainable params: 98,022
Non-trainable params: 2,996,352
_________________________________________________________________


In [7]:
callbacks = [
    keras.callbacks.EarlyStopping(
        patience=50, restore_best_weights=True),
    keras.callbacks.ModelCheckpoint(
        'best_model.keras', save_best_only=True),
]

In [ ]:
epochs = 10  # Adjust the number of epochs as needed

history = model.fit(
    ds_train,
    validation_data=ds_val,
    epochs=epochs,
    callbacks=callbacks
)

Epoch 1/10
97/97 [==============================] - 12s 82ms/step - loss: 2.5404 - accuracy: 0.4750 - val_loss: 1.4469 - val_accuracy: 0.7343
Epoch 2/10
97/97 [==============================] - 7s 71ms/step - loss: 0.7803 - accuracy: 0.8792 - val_loss: 0.8055 - val_accuracy: 0.8598
Epoch 3/10
86/97 [=========================>....] - ETA: 0s - loss: 0.4315 - accuracy: 0.9442

In [ ]:
def plot_history(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']

    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs_range = range(len(acc))

    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, acc, label='Training Accuracy')
    plt.plot(epochs_range, val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy')

    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, label='Training Loss')
    plt.plot(epochs_range, val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')
    plt.show()

plot_history(history)

### Evaluation

In [ ]:
test_loss, test_accuracy = model.evaluate(ds_test)
print(f'Test Accuracy: {test_accuracy:.4f}')